In [118]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd



# autoencoder
class Autoencoder(nn.Module):
  def __init__(self, input_dim, latent_dim):
      super().__init__()
      self.encoder = nn.Sequential(
           nn.Linear(input_dim, 256),
          nn.ReLU(),
          nn.Linear(256, 128),
          nn.ReLU(),
          nn.Linear(128, latent_dim)
        )
      self.decoder = nn.Sequential(
          nn.Linear(latent_dim, 128),
          nn.ReLU(),
          nn.Linear(128, 256),
          nn.ReLU(),            
          nn.Linear(256, input_dim)
        )
      self.centers = nn.Parameter(torch.randn(2, latent_dim))
    
  def forward(self, x):
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)        
      return encoded, decoded
    
  def get_center_loss(self, encoded, target):
      batch_size = encoded.size(0)
      target = target.reshape(-1, 1) #   view
      centers_batch = self.centers.gather(0, target.to(torch.int64).repeat(1, encoded.size(1))) 
      center_loss = (encoded - centers_batch).pow(2).sum() / batch_size
      return center_loss
  


# imbalanced에 data level로 해결하는 모델
class DDHS:
  #데이터를 KDE로 가우시안 분포를 이용하여 중간 %를 추출하는 함수
  # start, last에 퍼센트를 입력

  def extract_middle_percent(self,data, start, last):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(data_scaled)
    log_prob = kde.score_samples(data_scaled)
    prob = np.exp(log_prob)
    threshold_low, threshold_high = np.percentile(prob, [start, last])
    mask = np.logical_and(prob >= threshold_low, prob <= threshold_high)
    data_middle = data[mask]
    if len(data_middle) > 0 :
      return data_middle
    else:  
      print("No middle 50% found, returning original data")
      return np.array([])

  #  각 feature 안의 값을 복원추출하는 함수

  def reconstruct_features(self,data):
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    reconstructed = np.random.randn(*data.shape) * std + mean
    return reconstructed

  # synthetic sample을 생성하는 함수
  # 라벨과과 데이터를 하나의 데이터프레임으로 출력
  # small class / large class의 비율 = ratio 

  def generate_synthetic_sample(self,X,Y, ratio):

    data = pd.concat([X,Y],axis=1)

    # small class
    data_A = data[data[Y.columns[0]]== Y.value_counts().idxmin()[0]  ].loc[:, data.columns != Y.columns[0]].astype(float).values

    # large class 
    data_B = data[data[Y.columns[0]]== Y.value_counts().idxmax()[0] ].loc[:, data.columns != Y.columns[0]].astype(float).values


    # autoencoder를 사용하여 잠재 변수를 추출
    with torch.no_grad():
        encoded_A, _ = self.model(torch.tensor(data_A).float())
        encoded_B, _ = self.model(torch.tensor(data_B).float())

    # Majority : Q2 보다 Density가 큰 샘플을 Keep→ 50% 샘플을 Classifier 의 Train 데이터로 활용
    # Minority : Q3 을 사용해서 더 높은 기준을 설정 → 75% 샘플을 Classifier 의 Train 데이터로 활용 → 25% 샘플을 Subsequence 생성 과정의 샘플로 활용
  
    encoded_A_middle = self.extract_middle_percent(encoded_A.cpu().numpy(),12.5, 87.5) # 최종 합치기

    encoded_B_middle = self.extract_middle_percent(encoded_B.cpu().numpy(),25,75)# 최종 합치기

    # 중간 25%의 잠재 변수로부터 feature를 복원추출 -> 추가 확인인
    reconstructed_features = self.reconstruct_features(self.extract_middle_percent(encoded_A.cpu().numpy(),37.5, 62.5))

    # 임의의 위치에 synthetic sample 생성
    center_A = np.mean(encoded_A.cpu().numpy(), axis=0, dtype=np.float64, out=None)

    center_B = np.mean(encoded_B.numpy(), axis=0, dtype=np.float64, out=None) 

    radius_A = np.max(np.linalg.norm(encoded_A.cpu().numpy() - center_A, axis=1))

    #encoded_B[Y.columns[0]] = 1 
    synthetic_sample = pd.DataFrame() # 최종 합치기
   
    while (len(synthetic_sample)-len(encoded_B))/len(encoded_B) >= ratio :
        z = np.random.randn(latent_dim)
        if np.linalg.norm(z - center_A) < np.linalg.norm(z - center_B) and np.linalg.norm(z - center_A) < radius_A:
            synthetic_sample.append(z, ignore_index=True)

    # 최종 출력할 데이터 
    encoded_B_middle = pd.DataFrame(encoded_B_middle)
    encoded_B_middle['label'] = Y.value_counts().idxmin()[0]

    encoded_A_middle = pd.DataFrame(encoded_A_middle)
    encoded_A_middle['label'] = Y.value_counts().idxmax()[0] 

    synthetic_sample['label'] = Y.value_counts().idxmax()[0] 

    ouput = pd.concat([encoded_B_middle,encoded_A_middle,synthetic_sample ] )

    return ouput.loc[:, ouput.columns != 'label'],ouput['label']

  def fit(self,X,Y,lr,num_epochs, ratio):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_dim = len(X.columns) # 데이터의 차원
    latent_dim = len(X.columns) # 잠재 변수의 차원


    #lr = 1e-3 # 학습률
    #num_epochs = 50 # 학습 에폭 수
    
    self.model = Autoencoder(input_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)


    for epoch in range(num_epochs):
      x = torch.tensor(X.to_numpy()).float()#.to(device)
      y = torch.tensor(Y[Y.columns[0]].to_numpy()).float().to(device)

      encoded, decoded = self.model(x)
      reconstruction_loss = F.mse_loss(decoded, x)
      center_loss = self.model.get_center_loss(encoded, y)

      loss = reconstruction_loss + center_loss
      cross_entropy_loss = F.cross_entropy(decoded, y.long()) # y를 long 형으로 요구
      loss += cross_entropy_loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    
    synthetic_X, synthetic_Y = self.generate_synthetic_sample(X,Y, ratio)
    return synthetic_X, synthetic_Y


  def __init__(self):
    self.result = 0



In [119]:
magic = pd.read_csv('/content/drive/MyDrive/DDHS/magic04.data')
model = DDHS()

In [120]:
magic['g'].value_counts()

g    12331
h     6688
Name: g, dtype: int64

In [121]:
def digit_trans(x):
  if x == 'g':
    return 0
  else:
    return 1  
magic['label']=magic['g'].apply(digit_trans)

In [122]:
magic[['label']].columns[0]

'label'

In [123]:
magic[['label']].value_counts().idxmin()[0]

1

In [124]:
magic2= magic.drop('g',axis=1)
magic3 = magic2.drop('label',axis=1)

In [125]:
model.fit(magic3,magic[['label']], 1e-3, 50,1)

(              0          1         2          3          4          5  \
 0      5.355979  -0.887720  4.541102   9.841983  12.506713  -4.606334   
 1      5.341043  -2.410148  4.270937   8.797510  10.695181  -5.170622   
 2      2.470552  -0.513057  1.787970   4.505671   6.341879  -1.997583   
 3      7.936007  -4.786698  5.179059  11.967289  14.986421  -7.474792   
 4      7.489469  -4.904233  1.906083   5.128616   5.260751  -5.592462   
 ...         ...        ...       ...        ...        ...        ...   
 5011  10.863337  -6.895259  3.176227   3.892241   4.875031  -9.118051   
 5012   2.723989  -0.980387  1.897113   4.063401   5.136490  -2.295637   
 5013   7.352121  -1.871160  4.490939   6.575963  10.832455  -6.443671   
 5014   7.655222  -3.092923  3.163297   9.398839  11.834931  -6.198069   
 5015  23.298279 -19.666574 -2.194257  -2.419691  -1.307037 -16.106039   
 
              6         7          8          9  
 0    -1.999897  6.879706   8.466780  -5.326552  
 1    -1.4